In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
url = '/kaggle/input/mail-by/MAIL.csv'
df = pd.read_csv(url,header= None)
df_clean = df

In [ ]:
y = df_clean[0].values
X = df_clean.drop([0],axis = 1).values

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X, y)
allclass = np.unique(y_res)
allclassnumber = [sum(y_res==eachclass) for eachclass in allclass]
allclassnumber

In [ ]:
X_res.shape

In [ ]:
from sklearn.feature_selection import GenericUnivariateSelect
gn = GenericUnivariateSelect(mode='fdr', param=20)

In [ ]:
gn.fit(X_res,y_res)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
X_train = gn.transform(X_train)
X_test = gn.transform(X_test)
X_val = gn.transform(X_val)

In [ ]:
df_clean.shape

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm =  scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

In [ ]:
X_train_norm.shape

In [ ]:
import tensorflow as tf

In [ ]:
tf.compat.v1.reset_default_graph() # Clear Model
np.random.seed(1) # Fix Randomness
tf.compat.v1.random.set_random_seed(1) # Fix Randomness
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(500, activation='relu', input_shape=(400,)), #first layer[set input layer]
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(125, activation='relu'),
  tf.keras.layers.Dense(62.5, activation='relu'),
  tf.keras.layers.Dense(31.25, activation='relu'),
   tf.keras.layers.Dropout(0.2),#20%ของการกันสัญญาณวิ่ง
  tf.keras.layers.Dense(1, activation='sigmoid')    
])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_acc = list()
val_acc = list()
for i in range(0,500):
  history = model.fit(X_train_norm, y_train, epochs= 1, batch_size = 10, validation_data= (X_val_norm, y_val))
  tmp_avg = np.mean(history.history['accuracy'])
  tmp_avg_val = np.mean(history.history['val_accuracy'])
  train_acc.append(tmp_avg)
  val_acc.append(tmp_avg_val)

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.figure(num=None, figsize=(16, 8), dpi=90, facecolor='w', edgecolor='k')
plt.plot()
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_predict = np.round(model.predict(X_test_norm))
y_predict = [i[0] for i in y_predict.tolist()]
sum(y_predict == y_test)/len(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
target_names = ['negative', 'positive']
C = confusion_matrix(y_test,y_predict) 
C = C / C.astype(np.float).sum(axis=1)*100
sns.heatmap(C, annot=True, fmt=".2f",cmap="GnBu",xticklabels=target_names, yticklabels=target_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

blind

In [ ]:
url = '/kaggle/input/mail-test/MAIL_test.csv'
df_test = pd.read_csv(url,header= None)

In [ ]:
ytest = df_test[0].values
Xtest = df_test.drop(([0]),axis = 1).values

In [ ]:
Xtest = gn.transform(Xtest)

In [ ]:
X_test_norm = scaler.transform(Xtest)

In [ ]:
y_predict = np.round(model.predict(X_test_norm))
y_predict = [i[0] for i in y_predict.tolist()]
sum(y_predict == ytest)/len(ytest)